### Data munging to get response variables for lapse and withdrawal

In [1]:
raw <- readRDS('/data/capstone_data/ClusterData_allPHB_allvendorvars_editedcolumns.rds')

In [2]:
response_df <- raw[, c("Surr", "Efficiency_Category_Sub", "WDResponse")]

In [14]:
table(response_df$Surr)


      0       1 
4685495   47203 

In [15]:
table(response_df$Efficiency_Category_Sub)


     E      I     O1     O2     O3     U1     U2     U3 
519840 137400  71663  59541  84352  97670  82640  89552 

In [16]:
table(response_df$WDResponse)


      0       1 
3990289   50387 

In [17]:
write.csv(response_df, file = "/mnt/UW/outputDataset/response_3col.csv")

### Initiate h2o cluster to run glm with LASSO regularization

In [18]:
library(h2o)
h2o.init(nthreads = -1, max_mem_size = '400G')


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: ‘h2o’

The following object is masked from ‘package:RevoScaleR’:

    colnames

The following objects are masked from ‘package:stats’:

    cor, sd, var

The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc



 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 days 9 hours 
    H2O cluster version:        3.16.0.2 
    H2O cluster version age:    2 months and 23 days  
    H2O cluster name:           H2O_started_from_R_capsops_osa286 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   325.59 GB 
    H2O cluster total cores:    64 
    H2O cluster allowed cores:  64 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         XGBoost, Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.4.1 (2017-06-30) 



#### 1. Read in predictors from the cleaned data set

In [20]:
predictors <- h2o.importFile(path = normalizePath("/mnt/UW/outputDataset/allDataCleaned.csv")
                           , destination_frame = "predictors")

predictors$C1 <- NULL

  |======================================================================| 100%


In [21]:
response <- h2o.importFile(path = normalizePath("/mnt/UW/outputDataset/response_3col.csv")
                           , destination_frame = "response")

response$C1 <- NULL

  |======================================================================| 100%


In [22]:
full <- h2o.cbind(response, predictors)

In [32]:
full$Surr <- as.factor(full$Surr)

In [33]:
full$Efficiency_Category_Sub <- as.factor(full$Efficiency_Category_Sub)
full$WDResponse <- as.factor(full$WDResponse)

In [34]:
full.split <- h2o.splitFrame(full, ratios = c(0.5, 0.2), seed = -1)

In [99]:
training <- full.split[[1]]
validation <- full.split[[2]]

In [36]:
head(validation)

Surr,Efficiency_Category_Sub,WDResponse,PolNum_UW,JointInd,GMDBInd,OriginalOwner_C1,Qual,EligibleInd,FirstEligQInd,⋯,i12ccsb1_C4,i12ccsu1_C4,i12ccsv1_C4,i12ccsc1_C4,i12ccsd1_C4,i12ccsc2_C4,i03cctl1_C4,i12cctl1_C4,i12mtsq1_C4,i06txsb1_C4
0,NA,NA,86766,1,-1,-1,-1,NaN,NaN,⋯,0.2,0.0,0.0,0.00000,0,-0.500000,13.1000,100.300,0.0,0.0
0,NA,NA,476073,1,-1,-1,-1,NaN,NaN,⋯,19.6,1.2,-4.1,43.27141,0,0.100000,-30.9000,-22.200,0.0,0.2
0,NA,NA,179630,1,-1,NaN,-1,NaN,NaN,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NA,NA,500356,-1,-1,-1,-1,NaN,NaN,⋯,-0.7,-0.1,0.0,0.00000,0,0.000000,0.0000,2.800,0.0,0.0
0,NA,NA,204012,-1,-1,-1,-1,NaN,NaN,⋯,-34.1,-0.7,-1.0,29.90000,0,-1.303578,178.3204,318.139,0.0,0.0
0,NA,NA,481439,1,-1,-1,-1,NaN,NaN,⋯,0.0,0.0,-0.9,33.10000,0,1.100000,0.0000,8.900,0.1,0.0


h2o.glm(x, y, training_frame = , model_id, validation_frame = NULL,
  ignore_const_cols = TRUE, max_iterations = 50, beta_epsilon = 0,
  solver = c("IRLSM", "L_BFGS"), standardize = TRUE,
  family = c("gaussian", "binomial", "poisson", "gamma", "tweedie",
  "multinomial"), link = c("family_default", "identity", "logit", "log",
  "inverse", "tweedie"), tweedie_variance_power = NaN,
  tweedie_link_power = NaN, alpha = 0.5, prior = NULL, lambda = 1e-05,
  lambda_search = FALSE, nlambdas = -1, lambda_min_ratio = -1,
  nfolds = 0, fold_column = NULL, fold_assignment = c("AUTO", "Random",
  "Modulo"), keep_cross_validation_predictions = FALSE,
  beta_constraints = NULL, offset_column = NULL, weights_column = NULL,
  intercept = TRUE, max_active_predictors = -1, objective_epsilon = -1,
  gradient_epsilon = -1, non_negative = FALSE, compute_p_values = FALSE,
  remove_collinear_columns = FALSE, max_runtime_secs = 0,
  missing_values_handling = c("MeanImputation", "Skip"))

In [90]:
x <- names(full)[!names(full) %in% c("Surr", "Efficiency_Category_Sub", "WDResponse", "PolNum_UW", "AVPctEq",
                                     'JointInd', 'GMDBInd', 'Qual', 'EligibleInd', 'FirstEligQInd', 'UtilizationInd', 
                                     'Dist', 'Comm', 'Dur', 'SCPeriod', 'WDCount', 'AV', 'WDtoDate')]

#### 2. Lapse glm model with LASSO regularization

In [91]:
surr_fit <- h2o.glm(x, "Surr", training_frame = training, validation_frame = validation, family = "binomial", 
               alpha = 1, # LASSO
               standardize=TRUE,
               lambda_search = TRUE, nlambdas = 100, 
               lambda_min_ratio = 0.0001, nfolds = 0)

  |======================================================================| 100%


In [92]:
surr_fit@model$model_summary

family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
binomial,logit,Lasso (lambda = 2.259E-5 ),"nlambda = 100, lambda.max = 0.00454, lambda.min = 2.259E-5, lambda.1se = -1.0",514,313,67,RTMP_sid_9344_14


In [94]:
surr_var <- surr_fit@model$standardized_coefficient_magnitudes[1:50,]$names

In [95]:
surr_var

[1] "score_C4"                         "IssAgeALB"                       
 [3] "Household.Size.Indicator_C3.H"    "iat01_C4"                        
 [5] "iat02_C4"                         "iat33_C4"                        
 [7] "Presence.of.Kids.Indicator_C3.S"  "Nielsen.County.Size.Code_C3.A"   
 [9] "i12ccdu1_C4"                      "Home.Owner.Renter.Indicator_C3.H"
[11] "imt02y5_C4"                       "finscr_C4"                       
[13] "Number.of.Active.Sources_C3"      "imt20_C4"                        
[15] "imt01_C4"                         "curaddr_C4"                      
[17] "iat34_C4"                         "icc33_C4"                        
[19] "is004y5_C4"                       "CEN_bg_pctWorkforceForProfit"    
[21] "HealthScore_C5"                   "i12ccpq4_C4"                     
[23] "iat89_C4"                         "ifn01_C4"                        
[25] "ibr07_C4"                         "CEN_tr_pctWorkforceGovt"         
[27] "i12inpq4_C4"                      "CEN_tr_pctHHInvestIncome"        
[29] "icc21_C4"                         "imt07_C4"                        
[31] "imt96_C4"                         "i24txtb1_C4"                     
[33] "CEN_tr_pctHSGrad"                 "i12cctc1_C4"                     
[35] "imt01y5_C4"                       "i12cctq1_C4"                     
[37] "ire103_C4"                        "icc41_C4"                        
[39] "CEN_bg_pctHHincomeLT15K"          "Household.Size.Indicator_C3.S"   
[41] "CEN_tr_pctOwnOccValGE100K"        "OriginalOwner_C1"                
[43] "ihi20_C4"                         "CEN_bg_pctEducationIndustry"     
[45] "i03inpq4_C4"                      "iat51y5_C4"                      
[47] "i12cctl1_C4"                      "ifn102_C4"                       
[49] "Housing.Units.Percent.5.Units_C3" "CEN_tr_pctOwnOccSecondMort"

#### 3. Withdrawal efficiency group glm model with LASSO regularization

In [103]:
# Remove rows that are NA for 'Efficiency_Category_Sub' variable, because they are likely to be 
# policys that haven't started utilization.

training = training[training$Efficiency_Category_Sub != "NA",]
validation = validation[validation$Efficiency_Category_Sub != "NA",]
#h2o.table(validation$Efficiency_Category_Sub)

  Efficiency_Category_Sub  Count
1                       E 103636
2                       I  27432
3                      O1  14336
4                      O2  11908
5                      O3  16661
6                      U1  19662

[8 rows x 2 columns] 

In [104]:
eff_group_fit <- h2o.glm(x, "Efficiency_Category_Sub", training_frame = training, validation_frame = validation, family = "multinomial", 
                           alpha = 1, # LASSO
                           standardize=TRUE,
                           lambda_search = TRUE, nlambdas = 100, 
                           lambda_min_ratio = 0.0001, nfolds = 0)

  |======================================================================| 100%


In [105]:
eff_group_fit@model$model_summary

family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
multinomial,multinomial,Lasso (lambda = 6.531E-5 ),"nlambda = 100, lambda.max = 0.04827, lambda.min = 6.531E-5, lambda.1se = -1.0",4635,3022,78,RTMP_sid_9344_26


In [106]:
eff_group_var <- eff_group_fit@model$standardized_coefficient_magnitudes[1:50,]$names

In [107]:
eff_group_var

[1] "IssAgeALB"                        "score_C4"                        
 [3] "i12ccpc1_C4"                      "Household.Size.Indicator_C3.H"   
 [5] "iat52_C4"                         "finscr_C4"                       
 [7] "Gender.M"                         "i12ccpq4_C4"                     
 [9] "Target.Narrow.Band.Income_num"    "imt31_C4"                        
[11] "imt01_C4"                         "iau01y5_C4"                      
[13] "iat34_C4"                         "ihi20_C4"                        
[15] "Home.Owner.Renter.Indicator_C3.H" "iat01_C4"                        
[17] "HealthScore_C5"                   "i12ccnb1_C4"                     
[19] "iat34xam_C4"                      "ibr34_C4"                        
[21] "CEN_tr_pctHHincomeGE200K"         "is004_C4"                        
[23] "ibr33_C4"                         "imt41_C4"                        
[25] "iin01y5_C4"                       "ire103_C4"                       
[27] "i12ccpq2_C4"                      "Presence.of.Kids.Indicator_C3.S" 
[29] "imt02y5_C4"                       "iat07_C4"                        
[31] "CEN_bg_pctSeasonalHousingUnits"   "iat33_C4"                        
[33] "iat89_C4"                         "LotSize_C1"                      
[35] "ire31_C4"                         "Length.of.Residence_num"         
[37] "EstMarketValue_C1"                "ifn01y5_C4"                      
[39] "CEN_bg_pctConstructionIndustry"   "iat31xam_C4"                     
[41] "CEN_bg_pctLiveAloneHH"            "ihi01y5_C4"                      
[43] "Living.Area.Square.Feet_num"      "ifn01_C4"                        
[45] "imt01y5_C4"                       "CEN_tr_pctHHSocialSecurityIncome"
[47] "ibr41_C4"                         "CEN_tr_pctOwnOccValGE100K"       
[49] "Mortgage.Liability_C3"            "icc31_C4"

#### 4. What are the variables selected?

In [109]:
# Variable dictionary
df_vars <- read.csv('/home/capsops/mandy/selectedVariables.csv', encoding = 'latin1')

In [132]:
intersect(eff_group_var, surr_var)

[1] "IssAgeALB"                        "score_C4"                        
 [3] "Household.Size.Indicator_C3.H"    "finscr_C4"                       
 [5] "i12ccpq4_C4"                      "imt01_C4"                        
 [7] "iat34_C4"                         "ihi20_C4"                        
 [9] "Home.Owner.Renter.Indicator_C3.H" "iat01_C4"                        
[11] "HealthScore_C5"                   "ire103_C4"                       
[13] "Presence.of.Kids.Indicator_C3.S"  "imt02y5_C4"                      
[15] "iat33_C4"                         "iat89_C4"                        
[17] "ifn01_C4"                         "imt01y5_C4"                      
[19] "CEN_tr_pctOwnOccValGE100K"

In [133]:
lasso_features1 <- df_vars[df_vars$Variable %in% c(surr_var, eff_group_var),]

In [134]:
lasso_features2 <- df_vars[df_vars$Variable %in% intersect(surr_var, eff_group_var),]

In [138]:
write.csv(lasso_features1, '/mnt/UW/outputDataset/lassoFeatures1.csv')
write.csv(lasso_features1, 'lassoFeatures1.csv')

In [137]:
write.csv(lasso_features2, '/mnt/UW/outputDataset/lassoFeatures2.csv')
write.csv(lasso_features2, 'lassoFeatures2.csv')